### 本文档用于辅助理解项目各组成部分如何使用以及相关参数解释

In [ ]:
import asyncio, os
from oxygent import MAS, oxy,Config
Config.set_agent_llm_model("qwen")  # 设置默认的 llm 模型
LLM_MODEL = "qwen"  # Or any llm you can call

#### 工具定义

In [ ]:
file_tools = oxy.StdioMCPClient(
    name="文件管理系统",
    params={
        "command": "npx",
        "args": ["-y", "@modelcontextprotocol/server-filesystem", "./data"],## 使用npx启动 MCP 文件系统服务，数据目录为 ./data
    },
)
## 示例
oxy_my_tools = oxy.StdioMCPClient(  
name="my_tools",
    params={
        "command": "uv", 
        "args": ["--directory", "./mcp_servers", "run", "my_tools.py"], ## 启动一个 MCP 工具服务，它通过运行 uv --directory ./mcp_servers run my_tools.py 启动。
    },##启动后，agent就可以调用该工具的一些方法
) 

#### agent定义

In [ ]:
# 定义一个具有工具调用能力的 ReAct（Reason + Act）智能体，
# 用于根据问题推理并调用工具获取信息（如路径成本）。
tool_cost_agent = oxy.ReActAgent(
    name="tool_cost_agent",  
    # 智能体的名称，用于在多智能体系统中识别和调用。
    
    desc="The agent search the cost of path",
    # 描述该智能体的功能，帮助其他智能体或系统理解其用途。
    # 此处说明它用于查询路径的成本。
    
    tools=["my_tools/get_cost"],  
    # 指定该智能体可以使用的工具列表。
    # "my_tools/get_cost" 是一个自定义工具（通常在 oxy 工具注册系统中定义），
    # 用于根据输入（如起点、终点）返回路径成本。
    # 注意：工具名需与 oxy 中注册的工具名称一致。
    
    llm_model=LLM_MODEL,
    # 指定该智能体使用的底层大语言模型（如 "gpt-4", "claude-3", 或本地模型）。
    # LLM_MODEL 应已在代码其他地方定义，例如 LLM_MODEL = "gpt-4o"
)

# 定义一个专门用于生成计划（plan）的聊天智能体。
planner = oxy.ChatAgent(
    name="planner",
    # 智能体名称。

    llm_model=LLM_MODEL,
    # 使用的 LLM 模型。

    prompt=(
        "You are a planning agent.  "
        "Output **only** a JSON object that matches this schema:\n" 
        '{"steps": ["step 1", "step 2", ...]}\n'
    ),
    # 系统提示（system prompt），严格约束输出格式。
    # 要求该智能体只输出符合指定 JSON Schema 的对象，
    # 例如：{"steps": ["查询起点到终点的路径", "获取该路径的成本", "输出总成本"]}
    # 这种强格式约束便于后续程序解析。
)

# 定义一个“计划-执行”型智能体（Plan-and-Solve Agent），
# 它会先调用 planner 生成计划，再调用 executor 执行计划中的每一步。
analyser = oxy.PlanAndSolveAgent(
    name="analyser",
    # 智能体名称。

    llm_model=LLM_MODEL,
    # 使用的 LLM 模型（虽然 planner 和 executor 可能已有自己的模型，但此处可能用于协调或重试逻辑）。

    planner_agent_name=planner.name, 
    # 指定用于生成计划的 planner 智能体名称（即上面定义的 "planner"）。
    # PlanAndSolveAgent 会在内部调用该 planner 来获取步骤列表。

    executor_agent_name=executor.name,
    # 指定用于执行计划中每一步的 executor 智能体名称。
    # 注意：代码中未定义 executor，但应已存在，例如一个能调用 tool_cost_agent 的执行器。
    # executor 通常负责将 plan 中的自然语言步骤转化为具体动作（如调用工具）。

    pydantic_parser_planner=plan_parser,
    # 用于解析 planner 输出的 Pydantic 模型。
    # plan_parser 应是一个继承自 pydantic.BaseModel 的类，例如：
    #   class Plan(BaseModel):
    #       steps: List[str]
    # 这确保 planner 的输出被严格验证并转换为结构化对象。

    pydantic_parser_replanner=action_parser,
    # 用于在启用重规划（replanning）时解析新动作的 Pydantic 模型。
    # 即使当前未启用重规划，也可能需要提供（取决于框架实现）。

    enable_replanner=False,
    # 是否启用“重规划”机制。
    # 若为 True，当执行某步失败时，会调用 replanner 生成修正后的步骤。
    # 此处设为 False，表示执行失败时不会自动重规划，可能直接报错或停止。
)